In [1]:
#Desafío Manipulación y transformación de datos (Parte I)

#Ejercicio 1:

#0.Importar librería:
import pandas as pd

#a. Cargar los datos y crear un DataFrame con cada uno de ellos:
df_incidents = pd.read_pickle("incidents.pkl")
df_officers = pd.read_pickle("officers.pkl")
df_subjects = pd.read_pickle("subjects.pkl")

print(df_incidents.head())
print(df_officers.head())
print(df_subjects.head())

  case_number        date                  location subject_statuses  \
0      44523A  2013-02-23     3000 Chihuahua Street          Injured   
1     121982X  2010-05-03  1300 N. Munger Boulevard          Injured   
2     605484T  2007-08-12   200 S. Stemmons Freeway            Other   
3     384832T  2007-05-26           7900 S. Loop 12   Shoot and Miss   
4     244659R  2006-04-03        6512 South Loop 12          Injured   

  subject_weapon                                           subjects  \
0        Handgun                                   Curry, James L/M   
1        Handgun                                Chavez, Gabriel L/M   
2        Shotgun                                  Salinas, Nick L/M   
3        Unarmed  Smith, James B/M; Dews, Antonio B/M; Spearman,...   
4          Hands                                 Watkins, Caleb B/M   

   subject_count                                   officers  officer_count  \
0              1  Patino, Michael L/M; Fillingim, Brian W/M   

In [2]:
#b. Generar una tabla que contenga la unión de las 3 tablas:
df_merged = pd.merge(df_incidents, df_officers, on="case_number")
df_merged = pd.merge(df_merged, df_subjects, on="case_number", suffixes=("_officers", "_subjects"))
print(df_merged.head())

  case_number        date                  location subject_statuses  \
0      44523A  2013-02-23     3000 Chihuahua Street          Injured   
1      44523A  2013-02-23     3000 Chihuahua Street          Injured   
2     121982X  2010-05-03  1300 N. Munger Boulevard          Injured   
3     605484T  2007-08-12   200 S. Stemmons Freeway            Other   
4     384832T  2007-05-26           7900 S. Loop 12   Shoot and Miss   

  subject_weapon                                           subjects  \
0        Handgun                                   Curry, James L/M   
1        Handgun                                   Curry, James L/M   
2        Handgun                                Chavez, Gabriel L/M   
3        Shotgun                                  Salinas, Nick L/M   
4        Unarmed  Smith, James B/M; Dews, Antonio B/M; Spearman,...   

   subject_count                                   officers  officer_count  \
0              1  Patino, Michael L/M; Fillingim, Brian W/M   

In [3]:
#c. Verificar si hay filas duplicadas; si es así, elimínarlas:
duplicated_rows = df_merged[df_merged.duplicated()]
print(duplicated_rows)
#No hay filas duplicadas.

Empty DataFrame
Columns: [case_number, date, location, subject_statuses, subject_weapon, subjects, subject_count, officers, officer_count, grand_jury_disposition, attorney_general_forms_url, summary_url, summary_text, latitude, longitude, race_officers, gender_officers, last_name_officers, first_name_officers, full_name_officers, race_subjects, gender_subjects, last_name_subjects, first_name_subjects, full_name_subjects]
Index: []

[0 rows x 25 columns]


In [4]:
#d. ¿Cuántos sujetos de género F hay en el DataFrame resultante?:
numero_sujetos_f = df_merged["gender_subjects"].value_counts().get('F', 0)
print("Número de sujetos de género F en DataFrame resultante:", numero_sujetos_f)

Número de sujetos de género F en DataFrame resultante: 9


In [5]:
#e. ¿En cuántos números de caso hay por lo menos una sospechosa que sea mujer?:
casos_sospechosa = df_merged[df_merged["gender_subjects"] == 'F']['case_number'].nunique()
print("Número de casos en que por lo menos hay una sospechosa:", casos_sospechosa)

Número de casos en que por lo menos hay una sospechosa: 7


In [28]:
#f. Generar una tabla pivote que muestre el número de casos por mes en la fila y por año en las columnas:
#¿Se ve alguna relación entre mes del año y cantidad de casos?
df_merged["month"] = pd.to_datetime(df_merged["date"]).dt.month_name() 
df_merged["year"] = pd.to_datetime(df_merged["date"]).dt.year
tabla_pivote_casos = df_merged.pivot_table(index = df_merged["month"], columns = df_merged["year"], values = "case_number", aggfunc = "count")

print("\nTabla Pivote de Número de Casos por mes y año:\n")
print(tabla_pivote_casos)
print("\nRespuesta: No logro apreciar alguna relación clara entre el mes del año y la cantidad de casos.")



Tabla Pivote de Número de Casos por mes y año:

year       2003  2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  \
month                                                                         
April       1.0   4.0   NaN   2.0   NaN   1.0   NaN   1.0   NaN   NaN   3.0   
August      2.0   NaN   1.0   NaN   3.0   4.0   NaN   NaN   NaN   3.0   NaN   
December    NaN   3.0   4.0   NaN   NaN   2.0  15.0   2.0   5.0   3.0   1.0   
February    NaN   1.0   1.0   7.0   1.0   NaN   NaN   3.0   NaN   7.0   7.0   
January     2.0   2.0   NaN   1.0   1.0   1.0   2.0   NaN   NaN   NaN   7.0   
July        4.0   2.0   NaN   NaN   1.0   4.0   NaN   NaN   1.0   2.0   2.0   
June        8.0   1.0   NaN   NaN   3.0   3.0   NaN   2.0   2.0   2.0   3.0   
March       1.0   8.0   1.0   1.0   5.0   8.0   3.0   7.0   2.0   6.0   3.0   
May         1.0   1.0   2.0   2.0   9.0   5.0   4.0   1.0   1.0   9.0   1.0   
November    3.0   NaN   2.0   8.0   NaN   NaN   1.0   8.0   3.0   1.0   1.0   
Oct

In [27]:
#g. Generar una tabla pivote que muestre en las filas el género del oficial y en las columnas el género del subject.
#¿Cómo interpretas los valores que muestra esta vista?
tabla_pivote_genero = df_merged.pivot_table(index = "gender_officers", columns = "gender_subjects", values = "case_number", aggfunc = "count")

print("\nTabla Pivote de Género de Oficiales y Sospechosos:\n")
print(tabla_pivote_genero)
print("\nInterpretación:\n")
print("1. Del total de casos (377), 9 corresponden a sospechosos de género femenino (2,4%) y 368 a masculino (97,6%).")
print("2. De los casos relacionados a oficiales femeninas (20), 2 involucraron sospechosos de género femenino (10%) y 18 de masculino (90%).")
print("3. De los casos relacionados a oficiales masculinos (357), 7 involucraron sospechosos de género femenino (2%) y 350 de masculino (98%).")



Tabla Pivote de Género de Oficiales y Sospechosos:

gender_subjects  F    M
gender_officers        
F                2   18
M                7  350

Interpretación:

1. Del total de casos (377), 9 corresponden a sospechosos de género femenino (2,4%) y 368 a masculino (97,6%).
2. De los casos relacionados a oficiales femeninas (20), 2 involucraron sospechosos de género femenino (10%) y 18 de masculino (90%).
3. De los casos relacionados a oficiales masculinos (357), 7 involucraron sospechosos de género femenino (2%) y 350 de masculino (98%).


In [19]:
#Ejercicio 2:

#0.Importar librerías:
import numpy as np

#a. Cargar los datos y crear un DataFrame con ellos:
df_Cleaned_DS_Jobs = pd.read_csv("Cleaned_DS_Jobs.csv")

print(df_Cleaned_DS_Jobs.head())

           Job Title Salary Estimate  \
0  Sr Data Scientist        137-171    
1     Data Scientist        137-171    
2     Data Scientist        137-171    
3     Data Scientist        137-171    
4     Data Scientist        137-171    

                                     Job Description  Rating  \
0  Description\n\nThe Senior Data Scientist is re...     3.1   
1  Secure our Nation, Ignite your Future\n\nJoin ...     4.2   
2  Overview\n\n\nAnalysis Group is one of the lar...     3.8   
3  JOB DESCRIPTION:\n\nDo you have a passion for ...     3.5   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

         Company Name       Location            Headquarters  \
0         Healthfirst   New York, NY            New York, NY   
1             ManTech  Chantilly, VA             Herndon, VA   
2      Analysis Group     Boston, MA              Boston, MA   
3             INFICON     Newton, MA  Bad Ragaz, Switzerland   
4  Affinity Solutions   New York, NY            New Yo

In [20]:
#b. Utilizar la siguiente lista de valores que serán considerados como nulos:
#Listar valores nulos:
valores_nulos = ["na", "NA", -1, "0", "-1", "null", "n/a", "N/A", "NULL"]

#Reemplazar valores nulos por np.nan:
df_Cleaned_DS_Jobs.replace(valores_nulos, np.nan, inplace = True)

#Cantidad valores nulos por columna:
numero_valores_nulos = df_Cleaned_DS_Jobs.isna().sum()
print(numero_valores_nulos)

Job Title              0
Salary Estimate        0
Job Description        0
Rating                 0
Company Name           0
Location               0
Headquarters          31
Size                  27
Type of ownership     27
Industry              71
Sector                71
Revenue               27
min_salary             0
max_salary             0
avg_salary             0
job_state              0
same_state             0
company_age          118
python                 0
excel                  0
hadoop                 0
spark                  0
aws                    0
tableau                0
big_data               0
job_simp              68
seniority            567
dtype: int64


In [21]:
#c. Eliminar todas las filas con datos faltantes:

#Cantidad de filas antes de la eliminación:
numero_filas_antes = df_Cleaned_DS_Jobs.shape[0]
print("Número de filas antes de la eliminación:",numero_filas_antes)

#Eliminar filas con datos faltantes:
df_Cleaned_DS_Jobs.dropna(inplace=True)

#Cantidad de filas después de la eliminación:
numero_filas_despues = df_Cleaned_DS_Jobs.shape[0]
print("Número de filas después de la eliminación:",numero_filas_despues)

Número de filas antes de la eliminación: 660
Número de filas después de la eliminación: 80


In [22]:
#d. A partir de la columna “Salary Estimate”, generar dos columnas: Salario Estimado Mínimo y Máximo:

#Generar 2 columnas:
df_Cleaned_DS_Jobs[["Salario Estimado Mínimo", "Salario Estimado Máximo"]] = df_Cleaned_DS_Jobs["Salary Estimate"].str.split("-", expand = True)

#Convertir ambas columnas en valores numéricos:
df_Cleaned_DS_Jobs["Salario Estimado Mínimo"] = pd.to_numeric(df_Cleaned_DS_Jobs["Salario Estimado Mínimo"].str.strip(), errors = "coerce")
df_Cleaned_DS_Jobs["Salario Estimado Máximo"] = pd.to_numeric(df_Cleaned_DS_Jobs["Salario Estimado Máximo"].str.strip(), errors = "coerce")

print(df_Cleaned_DS_Jobs.head())

                                       Job Title Salary Estimate  \
0                              Sr Data Scientist        137-171    
32  Senior Research Statistician- Data Scientist         75-131    
38                 Senior Analyst/Data Scientist         75-131    
45                         Senior Data Scientist         75-131    
54                         Senior Data Scientist         75-131    

                                      Job Description  Rating  \
0   Description\n\nThe Senior Data Scientist is re...     3.1   
32  Acuity is seeking a Senior Research Statistici...     4.8   
38  At Edmunds were driven to make car buying easi...     3.4   
45  Klaviyo is looking for Senior Data Scientists ...     4.8   
54  Benson Hill empowers innovators to develop mor...     3.5   

        Company Name          Location      Headquarters  \
0        Healthfirst      New York, NY      New York, NY   
32  Acuity Insurance     Sheboygan, WI     Sheboygan, WI   
38       Edmunds.com

In [23]:
#e. Realizar la recodificación de la columna Size con los valores de la tabla entregada:

#Generar diccionario:
recodificación_size = { 
"10000+ employees": "Mega Empresas",
"5001 to 10000 employees": "Grandes Empresas",
"1001 to 5000 employees": "Medianas Empresas", 
"201 to 500 employees": "Pequeñas Empresas", 
"51 to 200 employees": "Pequeñas Grandes Empresas",
"501 to 1000 employees": "Microempresas",
"Unknown": "Empresas sin Información"
}

#Reemplazar valores:
df_Cleaned_DS_Jobs["Size"] = df_Cleaned_DS_Jobs["Size"].replace(recodificación_size)

print(df_Cleaned_DS_Jobs.head())

                                       Job Title Salary Estimate  \
0                              Sr Data Scientist        137-171    
32  Senior Research Statistician- Data Scientist         75-131    
38                 Senior Analyst/Data Scientist         75-131    
45                         Senior Data Scientist         75-131    
54                         Senior Data Scientist         75-131    

                                      Job Description  Rating  \
0   Description\n\nThe Senior Data Scientist is re...     3.1   
32  Acuity is seeking a Senior Research Statistici...     4.8   
38  At Edmunds were driven to make car buying easi...     3.4   
45  Klaviyo is looking for Senior Data Scientists ...     4.8   
54  Benson Hill empowers innovators to develop mor...     3.5   

        Company Name          Location      Headquarters               Size  \
0        Healthfirst      New York, NY      New York, NY  Medianas Empresas   
32  Acuity Insurance     Sheboygan, WI    

In [25]:
#f. Generar una tabla pivote que muestre la media del salario estimado mínimo y la media del salario estimado máximo por tamaño de empresa.

tabla_pivote_salario = df_Cleaned_DS_Jobs.pivot_table(index = "Size", values = ["Salario Estimado Mínimo", "Salario Estimado Máximo"], aggfunc = "mean")

print("\nTabla Pivote de Salarios Medios por Tamaño de Empresa:\n")
print(tabla_pivote_salario)


Tabla Pivote de Salarios Medios por Tamaño de Empresa:

                           Salario Estimado Máximo  Salario Estimado Mínimo
Size                                                                       
Empresas sin Información                110.500000                73.000000
Grandes Empresas                        138.875000                92.125000
Medianas Empresas                       137.461538                93.923077
Mega Empresas                           151.111111                97.888889
Microempresas                           146.235294               100.176471
Pequeñas Empresas                       141.142857                93.571429
Pequeñas Grandes Empresas               137.666667               100.666667
